# Importerer nødvendige biblioteker

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Indlæsning af data

In [4]:
df = pd.read_csv('../data/Data_Train.csv')
df.sample(5)

,Airline,Date_of_Journey,Source,Dep_Time,Destination,Class,Price
3833,Air India,6/03/2019,Mumbai,16:00,Hyderabad,0,11678
1008,Jet Airways,3/06/2019,Delhi,17:30,Cochin,0,10262
8478,Air India,6/03/2019,Mumbai,13:30,Hyderabad,0,18797
8007,Vistara,6/03/2019,Mumbai,15:40,Hyderabad,0,12395
8034,SpiceJet,1/03/2019,Mumbai,05:45,Hyderabad,0,12475


Vi har importeret dataene og lavet en hurtig gennemgang for at få et overblik over datasættet. Nu går vi videre med at forberede og rense dataene, så de er klar til analyse.

# Rensning af data

In [7]:
# Tjekker for manglende værdier
print("Tjekker for manglende værdier:")
print(df.isnull().sum())

# Tjekker for NaN-værdier
print("Tjekker for NaN-værdier:")
print(df.isna().sum())

# Tjekker for datatyper
print("Tjekker for datatyper:")
print(df.info())

# Tjekker for duplikater
print("Tjekker for duplikater:")
print(df.duplicated().sum()) 

Tjekker for manglende værdier:
Airline            0
Date_of_Journey    0
Source             0
Dep_Time           0
Destination        0
Class              0
Price              0
dtype: int64
Tjekker for NaN-værdier:
Airline            0
Date_of_Journey    0
Source             0
Dep_Time           0
Destination        0
Class              0
Price              0
dtype: int64
Tjekker for datatyper:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Dep_Time         10683 non-null  object
 4   Destination      10683 non-null  object
 5   Class            10683 non-null  int64 
 6   Price            10683 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 584.4+ KB
None
Tjekker for duplikater:
2001


In [8]:
df.shape

(10683, 7)

Som man kan se, indeholder datasættet ikke nogen manglende værdier, hverken som NaN eller null. Derudover består datatyperne primært af tekstdata (object), som nu skal konverteres til string, samt nogle numeriske kolonner i form af heltal (int64) og decimaltal (float64).

Der er duplikater i datasættet, men da disse kan repræsentere flere versioner af samme flyrejse (f.eks. forskellige priser afhængig af rute, dato eller flere premium-versioner for samme flyselskab), vælger vi at beholde dem i datasættet. Hvis vi fjerner duplikaterne, kan vi miste vigtige informationer om prisvariationer mellem f.eks. standard og premium-versioner af samme flyselskab. Flere premium-versioner kan eksistere for den samme rute og samme flyselskab, og derfor er det nødvendigt at have disse duplikater i datasættet for at få en korrekt analyse.

Derfor er der ikke behov for at fjerne duplikaterne.

Nu opdeles datasættet i to separate dataframes: en med numeriske værdier og en med nominelle (kategoriske) data, så det bliver lettere at arbejde målrettet med hver type.

In [10]:
# Fjerner 'Dep_Time' kolonnen, da den ikke er relevant for analysen
df.drop(['Dep_Time'], axis=1, inplace=True) 

In [11]:
# Konverterer 'Date_of_Journey' til datetime format
df['Date_of_Journey'] = pd.to_datetime(df['Date_of_Journey'], format='%d/%m/%Y') 

# Opretter en ny kolonne 'Journey_month' som indeholder måneden fra 'Date_of_Journey'
df['Journey_month'] = df['Date_of_Journey'].dt.month
df['Journey_month'] = df['Journey_month'].astype('int64')  # Skifter datatypen til int64

In [12]:
# Opdaterer den eksisterende 'Class' kolonne baseret på værdier i 'Airline' da den gamle kun havde 0 værdier
df['Class'] = df['Airline'].apply(lambda x: 'Business' if 'Business' in x else ('Premium economy' if 'Premium economy' in x else 'Standard'))

In [13]:
# Udfører one-hot encoding på de kategoriske kolonner for at gøre dem numeriske og lettere at regressere på
dfNumeric = pd.get_dummies(df, columns=['Airline', 'Source', 'Destination', 'Date_of_Journey'], dtype=pd.Int64Dtype())

# Viser 5 tilfældige rækker af det transformerede datasæt
dfNumeric.sample(5)

,Class,Price,Journey_month,Airline_Air Asia,Airline_Air India,Airline_GoAir,Airline_IndiGo,Airline_Jet Airways,Airline_Jet Airways Business,Airline_Multiple carriers,...,Date_of_Journey_2019-06-01 00:00:00,Date_of_Journey_2019-06-03 00:00:00,Date_of_Journey_2019-06-06 00:00:00,Date_of_Journey_2019-06-09 00:00:00,Date_of_Journey_2019-06-12 00:00:00,Date_of_Journey_2019-06-15 00:00:00,Date_of_Journey_2019-06-18 00:00:00,Date_of_Journey_2019-06-21 00:00:00,Date_of_Journey_2019-06-24 00:00:00,Date_of_Journey_2019-06-27 00:00:00
3479,Standard,5620,3,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7378,Standard,3815,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1286,Standard,8977,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4256,Standard,6896,5,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3699,Standard,8586,5,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
# Konverterer alle 'object'-kolonner til 'string'-type
for col in df:
    if df[col].dtype == 'object':
        df[col] = df[col].astype('string')

# Tjekker datatyperne efter konvertering
df.dtypes

Airline            string[python]
Date_of_Journey    datetime64[ns]
Source             string[python]
Destination        string[python]
Class              string[python]
Price                       int64
Journey_month               int64
dtype: object